In [ ]:
import pandas as pd       # Para manipulación de datos
import numpy as np        # Para trabajar con valores nulos y funciones numéricas

import seaborn as sns     # (opcional) Para visualizar valores nulos
import matplotlib.pyplot as plt  # (opcional) Para graficar distribución de datos


In [50]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install openpyxl


# INICIO DE CERO: ESIS

In [43]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importar librerías
import pandas as pd, os

# Definir rutas de entrada y salida
ruta_excel = '/content/drive/Shareddrives/analitica/datos_proyecto.xlsx'
ruta_salida = '/content/drive/Shareddrives/analitica/datos_proyecto_procesado.xlsx'

# Leer hoja específica (cambiar nombre de hoja aquí) y omitir filas extra
nombre_hoja = 'ESIS'  # <-- CAMBIA AQUÍ EL NOMBRE DE LA HOJA (CARRERA)
df = pd.read_excel(ruta_excel, sheet_name=nombre_hoja, skiprows=6)

# Convertir todo a texto y quitar espacios
for col in df.columns:
    df[col] = df[col].astype(str).str.strip()

# Reorganizar el DataFrame a formato largo (años como filas)
df_largo = df.melt(id_vars=['Puesto'], value_vars=list(range(2016, 2026)), var_name='Año', value_name='Puntaje')

# Limpiar valores: reemplazar vacíos y textos nulos por NaN
df_largo['Puntaje'] = df_largo['Puntaje'].replace(['', 'nan', 'NaN', 'None'], pd.NA)

# Convertir columnas a tipo adecuado, manteniendo errores
df_largo['Puesto'] = pd.to_numeric(df_largo['Puesto'], errors='coerce').astype('Int64')
df_largo['Año'] = pd.to_numeric(df_largo['Año'], errors='coerce').astype('Int64')

# Convertir puntaje a float, pero conservar 'N.S.P.'
df_largo['Puntaje'] = df_largo['Puntaje'].apply(lambda x: float(x) if str(x).replace('.', '', 1).isdigit() else x)

# Eliminar filas con valores nulos en 'Puesto' o 'Año', y conservar 'N.S.P.'
df_largo = df_largo.dropna(subset=['Puesto', 'Año'])
df_largo = df_largo[df_largo['Puntaje'].notna() | (df_largo['Puntaje'] == 'N.S.P.')]

# Leer los datos de ingresantes (fila 4, solo dos filas)
df_ingresantes = pd.read_excel(ruta_excel, sheet_name=nombre_hoja, skiprows=3, nrows=2)

# Extraer solo la fila que tiene números y convertirla en diccionario
ingresantes_dict = df_ingresantes.iloc[0, 1:].to_dict()

# Inicializar columna de ingreso como 'No'
df_largo['ingreso'] = 'No'

# Marcar como 'Si' a los primeros N puestos por año (según cantidad de ingresantes)
for year, cantidad in ingresantes_dict.items():
    mask = df_largo['Año'] == int(year)
    indices_ordenados = df_largo[mask].sort_values('Puesto').index
    df_largo.loc[indices_ordenados[:int(cantidad)], 'ingreso'] = 'Si'

# Exportar a Excel: si no existe, crear; si existe, reemplazar hoja correspondiente
# Determinar si el archivo ya existe
modo = 'a' if os.path.exists(ruta_salida) else 'w'

# Usar diferentes bloques según el modo
if modo == 'a':
    # Si existe, abre en modo append y reemplaza la hoja
    with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df_largo.to_excel(writer, sheet_name=f'{nombre_hoja}_Procesado', index=False)
else:
    # Si no existe, crea el archivo sin if_sheet_exists
    with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='w') as writer:
        df_largo.to_excel(writer, sheet_name=f'{nombre_hoja}_Procesado', index=False)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
# Obtener los nombres de las hojas
hojas = pd.ExcelFile(ruta_excel).sheet_names

# Mostrar los nombres
hojas

['ESIS', 'ESME', 'ESMC', 'ESMI', 'ESIQ', '21_25']

# EXPLICATIVO, se trabajará con hoja ESIS y se genera la hoja ESIS_ejemplo

In [51]:
# Obtener los nombres de las hojas
hojas = pd.ExcelFile(ruta_excel).sheet_names

# Mostrar los nombres
hojas

['ESIS', 'ESME', 'ESMC', 'ESMI', 'ESIQ', '21_25']

1. Conversión a texto y eliminación de espacios

In [53]:
# Leer hoja ESIS con filas omitidas
df_esis = pd.read_excel(ruta_excel, sheet_name='ESIS', skiprows=6)

df_esis.head()


,Puesto,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,1.0,380.0,381.0,442.0,445.375,233.250,400,318.00,395,457.75,424.25
1,2.0,360.0,360.0,340.0,362.000,212.125,388.75,315.25,386,456.50,406.50
2,3.0,350.0,357.0,320.0,317.500,203.250,388.75,297.25,364.25,448.75,406.25
3,4.0,350.0,356.0,300.0,311.875,201.000,388.75,256.50,364.25,442.25,386.00
4,5.0,320.0,331.0,296.0,306.500,201.000,377.5,236.00,356.5,441.00,384.75


In [55]:
# Convertir todas las columnas a texto y eliminar espacios
for col in df_esis.columns:
    df_esis[col] = df_esis[col].astype(str).str.strip()

df_esis

,Puesto,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,1.0,380.0,381.0,442.0,445.375,233.25,400,318.0,395,457.75,424.25
1,2.0,360.0,360.0,340.0,362.0,212.125,388.75,315.25,386,456.50,406.50
2,3.0,350.0,357.0,320.0,317.5,203.25,388.75,297.25,364.25,448.75,406.25
3,4.0,350.0,356.0,300.0,311.875,201.0,388.75,256.5,364.25,442.25,386.00
4,5.0,320.0,331.0,296.0,306.5,201.0,377.5,236.0,356.5,441.00,384.75
...,...,...,...,...,...,...,...,...,...,...,...
300,301.0,nan,nan,nan,nan,nan,NSP,41.25,nan,nan,nan
301,nan,nan,nan,nan,nan,nan,nan,33.5,nan,nan,nan
302,nan,nan,nan,nan,nan,nan,nan,32.25,nan,nan,nan
303,nan,nan,nan,nan,nan,nan,nan,0.0,nan,nan,nan


2. Reestructuración a formato largo

In [57]:
# Transformar columnas de años a filas (formato largo)
df_largo = df_esis.melt(id_vars=['Puesto'], value_vars=list(range(2016, 2026)), var_name='Año', value_name='Puntaje')

# Mostrar DataFrame transformado para captura
print("Datos en formato largo:")
df_largo

Datos en formato largo:


,Puesto,Año,Puntaje
0,1.0,2016,380.0
1,2.0,2016,360.0
2,3.0,2016,350.0
3,4.0,2016,350.0
4,5.0,2016,320.0
...,...,...,...
3045,301.0,2025,nan
3046,nan,2025,nan
3047,nan,2025,nan
3048,nan,2025,nan


3. TRATAMIENTO DE VALORES FALTANTES Y RUIDO

In [58]:
# Reemplazar valores vacíos o nulos comunes por pd.NA
df_largo['Puntaje'] = df_largo['Puntaje'].replace(['', 'nan', 'NaN', 'None'], pd.NA)

# Mostrar DataFrame tras reemplazo de valores nulos para captura
print("Datos después de tratar valores faltantes:")
df_largo

Datos después de tratar valores faltantes:


,Puesto,Año,Puntaje
0,1.0,2016,380.0
1,2.0,2016,360.0
2,3.0,2016,350.0
3,4.0,2016,350.0
4,5.0,2016,320.0
...,...,...,...
3045,301.0,2025,<NA>
3046,nan,2025,<NA>
3047,nan,2025,<NA>
3048,nan,2025,<NA>


4. Conversión segura de tipos numéricos

In [59]:
# Convertir Puesto y Año a entero (permitiendo NaN)
df_largo['Puesto'] = pd.to_numeric(df_largo['Puesto'], errors='coerce').astype('Int64')
df_largo['Año'] = pd.to_numeric(df_largo['Año'], errors='coerce').astype('Int64')

# Mostrar datos tras conversión de tipos para captura
print("Datos con 'Puesto' y 'Año' convertidos a enteros:")
df_largo

Datos con 'Puesto' y 'Año' convertidos a enteros:


,Puesto,Año,Puntaje
0,1,2016,380.0
1,2,2016,360.0
2,3,2016,350.0
3,4,2016,350.0
4,5,2016,320.0
...,...,...,...
3045,301,2025,<NA>
3046,<NA>,2025,<NA>
3047,<NA>,2025,<NA>
3048,<NA>,2025,<NA>


5. Conversión de puntaje a float con preservación de valores especiales

In [61]:
# Función para convertir a float o conservar texto especial
def convertir_puntaje(x):
    try:
        return float(x)
    except:
        return x

df_largo['Puntaje'] = df_largo['Puntaje'].apply(convertir_puntaje)

# Mostrar para captura los puntajes convertidos
df_largo

,Puesto,Año,Puntaje
0,1,2016,380.0
1,2,2016,360.0
2,3,2016,350.0
3,4,2016,350.0
4,5,2016,320.0
...,...,...,...
3045,301,2025,<NA>
3046,<NA>,2025,<NA>
3047,<NA>,2025,<NA>
3048,<NA>,2025,<NA>


6. Eliminación de filas incompletas
Se eliminan filas que no contienen información válida para 'Puesto' o 'Año', asegurando que sólo se analicen datos completos.

In [63]:
# Eliminar filas con valores nulos en 'Puesto' o 'Año'
df_largo = df_largo.dropna(subset=['Puesto', 'Año'])

# Eliminar filas donde 'Puntaje' es nulo, pero conservar si es 'N.S.P.'
df_largo = df_largo[df_largo['Puntaje'].notna() | (df_largo['Puntaje'] == 'N.S.P.')]

# Mostrar datos resultantes para captura
df_largo

,Puesto,Año,Puntaje
0,1,2016,380.0
1,2,2016,360.0
2,3,2016,350.0
3,4,2016,350.0
4,5,2016,320.0
...,...,...,...
2904,160,2025,86.0
2905,161,2025,56.75
2906,162,2025,N.S.P.
2907,163,2025,N.S.P.


7. Lectura de tabla de ingresantes

In [64]:
# Leer los datos de ingresantes (saltando primeras filas y leyendo solo dos)
df_ingresantes = pd.read_excel(ruta_excel, sheet_name='ESIS', skiprows=3, nrows=2)

# Mostrar tabla de cupos por año para captura
print("Cantidad de ingresantes por año:")
df_ingresantes

Cantidad de ingresantes por año:


,Año,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Cantidad de ingresantes:,24,12,17,19,31,47,46,28,12,12


8. Conversión de datos de ingresantes a diccionario

In [66]:
# Convertir la primera fila con datos a diccionario (sin incluir la columna de puesto)
ingresantes_dict = df_ingresantes.iloc[0, 1:].to_dict()

# Mostrar diccionario de ingresantes para ver cuántos cupos hay por año
print("Diccionario de cupos de ingreso por año:")
ingresantes_dict

Diccionario de cupos de ingreso por año:


{2016: 24,
 2017: 12,
 2018: 17,
 2019: 19,
 2020: 31,
 2021: 47,
 2022: 46,
 2023: 28,
 2024: 12,
 2025: 12}

9. Inicialización de columna de ingreso


In [68]:
# Inicializar columna con 'No'
df_largo['ingreso'] = 'No'

# Mostrar los primeros datos para ver la nueva columna
print("Columna 'ingreso' inicializada:")
df_largo


Columna 'ingreso' inicializada:


,Puesto,Año,Puntaje,ingreso
0,1,2016,380.0,No
1,2,2016,360.0,No
2,3,2016,350.0,No
3,4,2016,350.0,No
4,5,2016,320.0,No
...,...,...,...,...
2904,160,2025,86.0,No
2905,161,2025,56.75,No
2906,162,2025,N.S.P.,No
2907,163,2025,N.S.P.,No


10. Marcado de ingresantes por año según cupos


In [70]:
# Marcar como 'Si' a los primeros N puestos por año
for year, cantidad in ingresantes_dict.items():
    try:
        year = int(year)
        cantidad = int(cantidad)
        mask = df_largo['Año'] == year
        indices_ordenados = df_largo[mask].sort_values('Puesto').index
        df_largo.loc[indices_ordenados[:cantidad], 'ingreso'] = 'Si'
    except:
        continue  # En caso de errores, omitir año

# Mostrar resultado para captura
df_largo

,Puesto,Año,Puntaje,ingreso
0,1,2016,380.0,Si
1,2,2016,360.0,Si
2,3,2016,350.0,Si
3,4,2016,350.0,Si
4,5,2016,320.0,Si
...,...,...,...,...
2904,160,2025,86.0,No
2905,161,2025,56.75,No
2906,162,2025,N.S.P.,No
2907,163,2025,N.S.P.,No


11. Exportación a Excel (crea o reemplaza hoja)


In [72]:
# Determinar si el archivo ya existe
modo = 'a' if os.path.exists(ruta_salida) else 'w'

# Guardar el DataFrame con hoja nueva o reemplazada
if modo == 'a':
    with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df_largo.to_excel(writer, sheet_name='ESIS_ejemplo', index=False)
else:
    with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='w') as writer:
        df_largo.to_excel(writer, sheet_name='ESIS_ejemplo', index=False)

print(f"Archivo guardado correctamente en la hoja 'ESIS_ejemplo'")



Archivo guardado correctamente en la hoja 'ESIS_Procesado'


# ESME

In [45]:
nombre_hoja = 'ESME'  # <-- CAMBIA AQUÍ EL NOMBRE DE LA HOJA (CARRERA)
df = pd.read_excel(ruta_excel, sheet_name=nombre_hoja, skiprows=6)

# Convertir todo a texto y quitar espacios
for col in df.columns:
    df[col] = df[col].astype(str).str.strip()

# Reorganizar el DataFrame a formato largo (años como filas)
df_largo = df.melt(id_vars=['Puesto'], value_vars=list(range(2016, 2026)), var_name='Año', value_name='Puntaje')

# Limpiar valores: reemplazar vacíos y textos nulos por NaN
df_largo['Puntaje'] = df_largo['Puntaje'].replace(['', 'nan', 'NaN', 'None'], pd.NA)

# Convertir columnas a tipo adecuado, manteniendo errores
df_largo['Puesto'] = pd.to_numeric(df_largo['Puesto'], errors='coerce').astype('Int64')
df_largo['Año'] = pd.to_numeric(df_largo['Año'], errors='coerce').astype('Int64')

# Convertir puntaje a float, pero conservar 'N.S.P.'
df_largo['Puntaje'] = df_largo['Puntaje'].apply(lambda x: float(x) if str(x).replace('.', '', 1).isdigit() else x)

# Eliminar filas con valores nulos en 'Puesto' o 'Año', y conservar 'N.S.P.'
df_largo = df_largo.dropna(subset=['Puesto', 'Año'])
df_largo = df_largo[df_largo['Puntaje'].notna() | (df_largo['Puntaje'] == 'N.S.P.')]






# Leer los datos de ingresantes (fila 4, solo dos filas)
df_ingresantes = pd.read_excel(ruta_excel, sheet_name=nombre_hoja, skiprows=3, nrows=2)

# Extraer solo la fila que tiene números y convertirla en diccionario
ingresantes_dict = df_ingresantes.iloc[0, 1:].to_dict()

# Inicializar columna de ingreso como 'No'
df_largo['ingreso'] = 'No'

# Marcar como 'Si' a los primeros N puestos por año (según cantidad de ingresantes)
for year, cantidad in ingresantes_dict.items():
    mask = df_largo['Año'] == int(year)
    indices_ordenados = df_largo[mask].sort_values('Puesto').index
    df_largo.loc[indices_ordenados[:int(cantidad)], 'ingreso'] = 'Si'

# Exportar a Excel: si no existe, crear; si existe, reemplazar hoja correspondiente
# Determinar si el archivo ya existe
modo = 'a' if os.path.exists(ruta_salida) else 'w'

# Usar diferentes bloques según el modo
if modo == 'a':
    # Si existe, abre en modo append y reemplaza la hoja
    with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df_largo.to_excel(writer, sheet_name=f'{nombre_hoja}_Procesado', index=False)
else:
    # Si no existe, crea el archivo sin if_sheet_exists
    with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='w') as writer:
        df_largo.to_excel(writer, sheet_name=f'{nombre_hoja}_Procesado', index=False)


# ESMC

In [46]:
nombre_hoja = 'ESMC'  # <-- CAMBIA AQUÍ EL NOMBRE DE LA HOJA (CARRERA)
df = pd.read_excel(ruta_excel, sheet_name=nombre_hoja, skiprows=6)

# Convertir todo a texto y quitar espacios
for col in df.columns:
    df[col] = df[col].astype(str).str.strip()

# Reorganizar el DataFrame a formato largo (años como filas)
df_largo = df.melt(id_vars=['Puesto'], value_vars=list(range(2016, 2026)), var_name='Año', value_name='Puntaje')

# Limpiar valores: reemplazar vacíos y textos nulos por NaN
df_largo['Puntaje'] = df_largo['Puntaje'].replace(['', 'nan', 'NaN', 'None'], pd.NA)

# Convertir columnas a tipo adecuado, manteniendo errores
df_largo['Puesto'] = pd.to_numeric(df_largo['Puesto'], errors='coerce').astype('Int64')
df_largo['Año'] = pd.to_numeric(df_largo['Año'], errors='coerce').astype('Int64')

# Convertir puntaje a float, pero conservar 'N.S.P.'
df_largo['Puntaje'] = df_largo['Puntaje'].apply(lambda x: float(x) if str(x).replace('.', '', 1).isdigit() else x)

# Eliminar filas con valores nulos en 'Puesto' o 'Año', y conservar 'N.S.P.'
df_largo = df_largo.dropna(subset=['Puesto', 'Año'])
df_largo = df_largo[df_largo['Puntaje'].notna() | (df_largo['Puntaje'] == 'N.S.P.')]

# Leer los datos de ingresantes (fila 4, solo dos filas)
df_ingresantes = pd.read_excel(ruta_excel, sheet_name=nombre_hoja, skiprows=3, nrows=2)

# Extraer solo la fila que tiene números y convertirla en diccionario
ingresantes_dict = df_ingresantes.iloc[0, 1:].to_dict()

# Inicializar columna de ingreso como 'No'
df_largo['ingreso'] = 'No'

# Marcar como 'Si' a los primeros N puestos por año (según cantidad de ingresantes)
for year, cantidad in ingresantes_dict.items():
    mask = df_largo['Año'] == int(year)
    indices_ordenados = df_largo[mask].sort_values('Puesto').index
    df_largo.loc[indices_ordenados[:int(cantidad)], 'ingreso'] = 'Si'

# Exportar a Excel: si no existe, crear; si existe, reemplazar hoja correspondiente
# Determinar si el archivo ya existe
modo = 'a' if os.path.exists(ruta_salida) else 'w'

# Usar diferentes bloques según el modo
if modo == 'a':
    # Si existe, abre en modo append y reemplaza la hoja
    with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df_largo.to_excel(writer, sheet_name=f'{nombre_hoja}_Procesado', index=False)
else:
    # Si no existe, crea el archivo sin if_sheet_exists
    with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='w') as writer:
        df_largo.to_excel(writer, sheet_name=f'{nombre_hoja}_Procesado', index=False)


# ESMI

In [47]:
nombre_hoja = 'ESMI'  # <-- CAMBIA AQUÍ EL NOMBRE DE LA HOJA (CARRERA)
df = pd.read_excel(ruta_excel, sheet_name=nombre_hoja, skiprows=6)

# Convertir todo a texto y quitar espacios
for col in df.columns:
    df[col] = df[col].astype(str).str.strip()

# Reorganizar el DataFrame a formato largo (años como filas)
df_largo = df.melt(id_vars=['Puesto'], value_vars=list(range(2016, 2026)), var_name='Año', value_name='Puntaje')

# Limpiar valores: reemplazar vacíos y textos nulos por NaN
df_largo['Puntaje'] = df_largo['Puntaje'].replace(['', 'nan', 'NaN', 'None'], pd.NA)

# Convertir columnas a tipo adecuado, manteniendo errores
df_largo['Puesto'] = pd.to_numeric(df_largo['Puesto'], errors='coerce').astype('Int64')
df_largo['Año'] = pd.to_numeric(df_largo['Año'], errors='coerce').astype('Int64')

# Convertir puntaje a float, pero conservar 'N.S.P.'
df_largo['Puntaje'] = df_largo['Puntaje'].apply(lambda x: float(x) if str(x).replace('.', '', 1).isdigit() else x)

# Eliminar filas con valores nulos en 'Puesto' o 'Año', y conservar 'N.S.P.'
df_largo = df_largo.dropna(subset=['Puesto', 'Año'])
df_largo = df_largo[df_largo['Puntaje'].notna() | (df_largo['Puntaje'] == 'N.S.P.')]

# Leer los datos de ingresantes (fila 4, solo dos filas)
df_ingresantes = pd.read_excel(ruta_excel, sheet_name=nombre_hoja, skiprows=3, nrows=2)

# Extraer solo la fila que tiene números y convertirla en diccionario
ingresantes_dict = df_ingresantes.iloc[0, 1:].to_dict()

# Inicializar columna de ingreso como 'No'
df_largo['ingreso'] = 'No'

# Marcar como 'Si' a los primeros N puestos por año (según cantidad de ingresantes)
for year, cantidad in ingresantes_dict.items():
    mask = df_largo['Año'] == int(year)
    indices_ordenados = df_largo[mask].sort_values('Puesto').index
    df_largo.loc[indices_ordenados[:int(cantidad)], 'ingreso'] = 'Si'

# Exportar a Excel: si no existe, crear; si existe, reemplazar hoja correspondiente
# Determinar si el archivo ya existe
modo = 'a' if os.path.exists(ruta_salida) else 'w'

# Usar diferentes bloques según el modo
if modo == 'a':
    # Si existe, abre en modo append y reemplaza la hoja
    with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df_largo.to_excel(writer, sheet_name=f'{nombre_hoja}_Procesado', index=False)
else:
    # Si no existe, crea el archivo sin if_sheet_exists
    with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='w') as writer:
        df_largo.to_excel(writer, sheet_name=f'{nombre_hoja}_Procesado', index=False)


# ESIQ

In [48]:
nombre_hoja = 'ESIQ'  # <-- CAMBIA AQUÍ EL NOMBRE DE LA HOJA (CARRERA)
df = pd.read_excel(ruta_excel, sheet_name=nombre_hoja, skiprows=6)

# Convertir todo a texto y quitar espacios
for col in df.columns:
    df[col] = df[col].astype(str).str.strip()

# Reorganizar el DataFrame a formato largo (años como filas)
df_largo = df.melt(id_vars=['Puesto'], value_vars=list(range(2016, 2026)), var_name='Año', value_name='Puntaje')

# Limpiar valores: reemplazar vacíos y textos nulos por NaN
df_largo['Puntaje'] = df_largo['Puntaje'].replace(['', 'nan', 'NaN', 'None'], pd.NA)

# Convertir columnas a tipo adecuado, manteniendo errores
df_largo['Puesto'] = pd.to_numeric(df_largo['Puesto'], errors='coerce').astype('Int64')
df_largo['Año'] = pd.to_numeric(df_largo['Año'], errors='coerce').astype('Int64')

# Convertir puntaje a float, pero conservar 'N.S.P.'
df_largo['Puntaje'] = df_largo['Puntaje'].apply(lambda x: float(x) if str(x).replace('.', '', 1).isdigit() else x)

# Eliminar filas con valores nulos en 'Puesto' o 'Año', y conservar 'N.S.P.'
df_largo = df_largo.dropna(subset=['Puesto', 'Año'])
df_largo = df_largo[df_largo['Puntaje'].notna() | (df_largo['Puntaje'] == 'N.S.P.')]

# Leer los datos de ingresantes (fila 4, solo dos filas)
df_ingresantes = pd.read_excel(ruta_excel, sheet_name=nombre_hoja, skiprows=3, nrows=2)

# Extraer solo la fila que tiene números y convertirla en diccionario
ingresantes_dict = df_ingresantes.iloc[0, 1:].to_dict()

# Inicializar columna de ingreso como 'No'
df_largo['ingreso'] = 'No'

# Marcar como 'Si' a los primeros N puestos por año (según cantidad de ingresantes)
for year, cantidad in ingresantes_dict.items():
    mask = df_largo['Año'] == int(year)
    indices_ordenados = df_largo[mask].sort_values('Puesto').index
    df_largo.loc[indices_ordenados[:int(cantidad)], 'ingreso'] = 'Si'

# Exportar a Excel: si no existe, crear; si existe, reemplazar hoja correspondiente
# Determinar si el archivo ya existe
modo = 'a' if os.path.exists(ruta_salida) else 'w'

# Usar diferentes bloques según el modo
if modo == 'a':
    # Si existe, abre en modo append y reemplaza la hoja
    with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df_largo.to_excel(writer, sheet_name=f'{nombre_hoja}_Procesado', index=False)
else:
    # Si no existe, crea el archivo sin if_sheet_exists
    with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='w') as writer:
        df_largo.to_excel(writer, sheet_name=f'{nombre_hoja}_Procesado', index=False)
